In [1]:
import pandas as pd

# read data

In [2]:
#Change the restaurant excel file name here
excel_sheet_name = "excel sheet 3.xlsx"

#name of the legend excel file
legend_sheet_name =  "legend.xlsx"

#enter the name of the excel file result
result_excel = "./excel sheet 4.xlsx"



In [3]:
def import_data(excel_sheet_name,legend_sheet_name):
    xls_file = pd.ExcelFile(excel_sheet_name)
    sheets = xls_file.sheet_names
    sides = pd.DataFrame()
    desserts = pd.DataFrame()
    single_items = pd.read_excel(excel_sheet_name,sheet_name='single items')
    if 'sides' in sheets:
        sides = pd.read_excel(excel_sheet_name,sheet_name='sides')
    if 'drinks-desserts' in sheets:
        desserts = pd.read_excel(excel_sheet_name,sheet_name='drinks-desserts')
    legend = pd.read_excel(legend_sheet_name,sheet_name='legend')
    return single_items,sides,desserts,legend

In [4]:
single_items,sides,desserts,legend = import_data(excel_sheet_name,legend_sheet_name)

In [5]:
single_items.head(3)

,menu,Restaurant Price,Recipe Cost,Research links,Meal recipe
0,Jumbo Burrito,12.42,3.29,NaN,$0.5 marinated steak\n$0.1 2oz refried beans\n...
1,California Burrito,12.64,4.00,NaN,$0.5 marinated steak\n$1.17 waffle fries\n$0.3...
2,Super Burrito,14.29,5.72,NaN,$1 marinated steak\n$1.17 waffle fries\n$0.35 ...


# prepare the analytics algorithms

In [6]:
#Filling the missing values with 0
def filling_null_values(data):
    data['Recipe Cost'] = data['Recipe Cost'].fillna(0)
    data['Restaurant Price'] = data['Restaurant Price'].fillna(0)
    return data

In [7]:
def creating_recipe_research(test_single_items,test_sides,test_drinks_desserts):
    single_items_recipes = test_single_items.drop(['Restaurant Price','Recipe Cost'], axis=1)
    test_single_items = test_single_items.drop(['Research links','Meal recipe'], axis=1)
    if len(test_sides)!=0:
        sides_recipes =  test_sides.drop(['Restaurant Price','Recipe Cost'], axis=1)
        test_sides = test_sides.drop(['Research links','Meal recipe'], axis=1)
        single_items_recipes = pd.concat([single_items_recipes, sides_recipes], axis=0)
    if len(test_drinks_desserts)!=0:
        desserts_recipes =  test_drinks_desserts.drop(['Restaurant Price','Recipe Cost'], axis=1)
        test_drinks_desserts = test_drinks_desserts.drop(['Research links','Meal recipe'], axis=1)
        single_items_recipes = pd.concat([single_items_recipes, desserts_recipes], axis=0)
    return single_items_recipes,test_single_items,test_sides,test_drinks_desserts

In [8]:
#Restaurant price after food costs
def restaurant_price_after_food_costs(data):
    data['Restaurant price after food costs'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Restaurant Price'][i] != 0) & (data['Recipe Cost'][i]!=0):
            data['Restaurant price after food costs'][i] = data['Restaurant Price'][i] - data['Recipe Cost'][i] 
    return data

In [9]:
#Restaurant price after 40% fixed costs
def restaurant_price_after_fixed_costs(data):
    data['Restaurant price after 40% fixed costs'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Restaurant Price'][i] != 0):
            fixed_cost = data['Restaurant Price'][i] * 0.4
            data['Restaurant price after 40% fixed costs'][i] = data['Restaurant Price'][i] - fixed_cost
    return data

In [10]:
#Restaurant price after food costs and 40% fixed costs
def restaurant_price_after_food_costs_and_40fixed_costs(data):
    data['Restaurant price after food and 40% fixed costs'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Restaurant Price'][i] != 0) & (data['Recipe Cost'][i]!=0):
            fixed_cost = data['Restaurant Price'][i] * 0.4
            data['Restaurant price after food and 40% fixed costs'][i] = data['Restaurant Price'][i] - (data['Recipe Cost'][i] + fixed_cost)
    return data

In [11]:
#item's price after FH discount
def price_after_FH(data):
    data['Price after FH discount'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Restaurant Price'][i] != 0):
            data['Price after FH discount'][i] = data['Restaurant Price'][i]*0.6
    return data

In [12]:
#profit after 40fixed_cost and FH discount
def profit_after_40fixed_cost_and_FH_discount(data):
    data['Profit after 40% fixed cost and FH discount'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Price after FH discount'][i] != 0) & (data['Recipe Cost'][i]!=0):
            fixed_cost = data['Restaurant Price'][i] * 0.4
            data['Profit after 40% fixed cost and FH discount'][i] = data['Price after FH discount'][i]-(data['Recipe Cost'][i]+fixed_cost)
    return data

In [13]:
#profit margin after 40fixed_cost and FH discount
def profit_margin_after_40fixed_cost_and_FH_discount(data):
    data['Profit margin after 40% fixed cost and FH discount'] = 0.0
    for i,row in data.iterrows(): 
        if (data['Price after FH discount'][i] != 0) & (data['Profit after 40% fixed cost and FH discount'][i]!=0):
            data['Profit margin after 40% fixed cost and FH discount'][i] = data['Profit after 40% fixed cost and FH discount'][i]/data['Price after FH discount'][i]
    return data

In [14]:
single_items.head()

,menu,Restaurant Price,Recipe Cost,Research links,Meal recipe
0,Jumbo Burrito,12.42,3.29,NaN,$0.5 marinated steak\n$0.1 2oz refried beans\n...
1,California Burrito,12.64,4.00,NaN,$0.5 marinated steak\n$1.17 waffle fries\n$0.3...
2,Super Burrito,14.29,5.72,NaN,$1 marinated steak\n$1.17 waffle fries\n$0.35 ...
3,Regular Burrito,10.99,1.64,NaN,$0.1 2oz refried beans\n$0.36 2oz rice\n$1.18 ...
4,Vegetarian Burrito,10.44,3.29,NaN,$0.1 2oz refried beans\n$0.36 2oz rice\n$1.18 ...


In [15]:
#getting all research links and recipes in a new sheet
recipes,single_items,sides,desserts = creating_recipe_research(single_items,sides,desserts)


#profit margin calculation before and after FH discount for single items
single_items = filling_null_values(single_items)
single_items = restaurant_price_after_food_costs(single_items)
single_items = restaurant_price_after_fixed_costs(single_items)
single_items = restaurant_price_after_food_costs_and_40fixed_costs(single_items)

single_items = price_after_FH(single_items)

single_items = profit_after_40fixed_cost_and_FH_discount(single_items)
single_items = profit_margin_after_40fixed_cost_and_FH_discount(single_items)



#profit margin calculation before and after FH discount for sides
if len(sides)!=0:
    sides = filling_null_values(sides)
    sides = restaurant_price_after_food_costs(sides)
    sides = restaurant_price_after_fixed_costs(sides)
    sides = restaurant_price_after_food_costs_and_40fixed_costs(sides)

    sides = price_after_FH(sides)

    sides = profit_after_40fixed_cost_and_FH_discount(sides)
    sides = profit_margin_after_40fixed_cost_and_FH_discount(sides)


#profit margin calculation before and after FH discount for drinks-desserts
if len(desserts)!=0:
    desserts = filling_null_values(desserts)
    desserts = restaurant_price_after_food_costs(desserts)
    desserts = restaurant_price_after_fixed_costs(desserts)
    desserts = restaurant_price_after_food_costs_and_40fixed_costs(desserts)

    desserts = price_after_FH(desserts)

    desserts = profit_after_40fixed_cost_and_FH_discount(desserts)
    desserts = profit_margin_after_40fixed_cost_and_FH_discount(desserts)


C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/111061102.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Restaurant price after food costs'][i] = data['Restaurant Price'][i] - data['Recipe Cost'][i]
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/1814551043.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Restaurant price after 40% fixed costs'][i] = data['Restaurant Price'][i] - fixed_cost
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/3833067688.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [16]:
recipes.head()

,menu,Research links,Meal recipe
0,Jumbo Burrito,NaN,$0.5 marinated steak\n$0.1 2oz refried beans\n...
1,California Burrito,NaN,$0.5 marinated steak\n$1.17 waffle fries\n$0.3...
2,Super Burrito,NaN,$1 marinated steak\n$1.17 waffle fries\n$0.35 ...
3,Regular Burrito,NaN,$0.1 2oz refried beans\n$0.36 2oz rice\n$1.18 ...
4,Vegetarian Burrito,NaN,$0.1 2oz refried beans\n$0.36 2oz rice\n$1.18 ...


# prepare the meal combinations algorithms

In [17]:
def meals_creation_all_items(single_items,sides,desserts):
    menus = pd.DataFrame(columns=['menu',  'Restaurant Price', 'Recipe Cost'])
    for i,item in single_items.iterrows():
        item_name = single_items['menu'][i]
        item_price = single_items['Restaurant Price'][i]
        item_cost = single_items['Recipe Cost'][i]
        for j,side in sides.iterrows():
            side_name = sides['menu'][j]
            side_price = sides['Restaurant Price'][j]
            side_cost = sides['Recipe Cost'][j]  
            for s,dessert in desserts.iterrows():
                dessert_name = desserts['menu'][s]
                dessert_price = desserts['Restaurant Price'][s]
                dessert_cost = desserts['Recipe Cost'][s]
                new_menu_name = str(item_name)+'+'+str(side_name)+'+'+str(dessert_name)
                new_menu_price = item_price + side_price + dessert_price
                new_menu_cost = item_cost + side_cost +dessert_cost
                menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
    return menus

In [18]:
def meals_creation_no_sides(single_items,desserts):
    menus = pd.DataFrame(columns=['menu',  'Restaurant Price', 'Recipe Cost'])
    for i,item in single_items.iterrows():
        item_name = single_items['menu'][i]
        item_price = single_items['Restaurant Price'][i]
        item_cost = single_items['Recipe Cost'][i]
        for s,dessert in desserts.iterrows():
            dessert_name = desserts['menu'][s]
            dessert_price = desserts['Restaurant Price'][s]
            dessert_cost = desserts['Recipe Cost'][s]
            new_menu_name = str(item_name)+'+'+str(dessert_name)
            new_menu_price = item_price + dessert_price
            new_menu_cost = item_cost +dessert_cost
            menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
    return menus

In [19]:
def meals_creation_no_desserts(single_items,sides):
    menus = pd.DataFrame(columns=['menu',  'Restaurant Price', 'Recipe Cost'])
    for i,item in single_items.iterrows():
        item_name = single_items['menu'][i]
        item_price = single_items['Restaurant Price'][i]
        item_cost = single_items['Recipe Cost'][i]
        for j,side in sides.iterrows():
            side_name = sides['menu'][j]
            side_price = sides['Restaurant Price'][j]
            side_cost = sides['Recipe Cost'][j]
            
            new_menu_name = str(item_name)+'+'+str(side_name)
            new_menu_price = item_price + side_price
            new_menu_cost = item_cost + side_cost
            menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
            
    return menus

In [20]:
menus = pd.DataFrame(columns=['menu',  'Restaurant Price', 'Recipe Cost'])
if (sides.empty) & len(desserts)!=0:
    menus = meals_creation_no_sides(single_items,desserts)
elif desserts.empty & len(sides)!=0:
    menus = meals_creation_no_desserts(single_items,sides)
else:
    menus = meals_creation_all_items(single_items,sides,desserts)

if len(menus)!=0:
    menus = restaurant_price_after_food_costs(menus)
    menus =restaurant_price_after_fixed_costs(menus)
    menus = restaurant_price_after_food_costs_and_40fixed_costs(menus)

    menus = price_after_FH(menus)

    menus = profit_after_40fixed_cost_and_FH_discount(menus)
    menus = profit_margin_after_40fixed_cost_and_FH_discount(menus)
    menus = menus.sort_values(by=['Profit margin after 40% fixed cost and FH discount'], ascending=False)

single_items = single_items.sort_values(by=['Profit margin after 40% fixed cost and FH discount'], ascending=False)
if len(sides)!=0:
    sides = sides.sort_values(by=['Profit margin after 40% fixed cost and FH discount'], ascending=False)
if len(desserts)!=0:
    desserts = desserts.sort_values(by=['Profit margin after 40% fixed cost and FH discount'], ascending=False)
    

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

...

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=True)
C:\Users\dridi\AppData\Local\Temp/ipykernel_22188/2563300326.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  menus = menus.append({'menu': new_menu_name, 'Restaurant Price' : new_menu_price,'Recipe Cost' : new_menu_cost}, ignore_index=T

In [26]:
menus.head(3)

,menu,Restaurant Price,Recipe Cost,Restaurant price after food costs,Restaurant price after 40% fixed costs,Restaurant price after food and 40% fixed costs,Price after FH discount,Profit after 40% fixed cost and FH discount,Profit margin after 40% fixed cost and FH discount
1213,Taco Salad+Side Refried Beans & Rice+Bottled W...,19.12,1.9,17.22,11.472,9.572,11.472,1.924,0.167713
1214,Taco Salad+Side Refried Beans & Rice+Arizona,19.12,2.03,17.09,11.472,9.442,11.472,1.794,0.156381
1195,Taco Salad+Jalapenos+Bottled Water,16.01,1.84,14.17,9.606,7.766,9.606,1.362,0.141786


In [22]:
single_items.head(3)

,menu,Restaurant Price,Recipe Cost,Restaurant price after food costs,Restaurant price after 40% fixed costs,Restaurant price after food and 40% fixed costs,Price after FH discount,Profit after 40% fixed cost and FH discount,Profit margin after 40% fixed cost and FH discount
28,Taco Salad,12.53,1.29,11.24,7.518,6.228,7.518,1.216,0.161745
26,Regular Nacho,10.44,1.55,8.89,6.264,4.714,6.264,0.538,0.085888
3,Regular Burrito,10.99,1.64,9.35,6.594,4.954,6.594,0.558,0.084622


In [23]:
sides.head(3)

,menu,Restaurant Price,Recipe Cost,Restaurant price after food costs,Restaurant price after 40% fixed costs,Restaurant price after food and 40% fixed costs,Price after FH discount,Profit after 40% fixed cost and FH discount,Profit margin after 40% fixed cost and FH discount
6,Side Refried Beans & Rice,4.40,0.46,3.94,2.640,2.180,2.640,0.420,0.159091
3,Jalapenos,1.29,0.40,0.89,0.774,0.374,0.774,-0.142,-0.183463
5,Chips & Salsa,2.75,0.90,1.85,1.650,0.750,1.650,-0.350,-0.212121


In [24]:
desserts.head(3)

,menu,Restaurant Price,Recipe Cost,Restaurant price after food costs,Restaurant price after 40% fixed costs,Restaurant price after food and 40% fixed costs,Price after FH discount,Profit after 40% fixed cost and FH discount,Profit margin after 40% fixed cost and FH discount
1,Bottled Water,2.19,0.15,2.04,1.314,1.164,1.314,0.288,0.219178
2,Arizona,2.19,0.28,1.91,1.314,1.034,1.314,0.158,0.120244
0,Can Soft Drink 12 Oz,2.19,0.48,1.71,1.314,0.834,1.314,-0.042,-0.031963


# save results

In [25]:
#save results
writer = pd.ExcelWriter(result_excel, engine = 'xlsxwriter')
legend.to_excel(writer, sheet_name = 'legend',index=False)
recipes.to_excel(writer, sheet_name = 'recipe research',index=False)
single_items.to_excel(writer, sheet_name = 'single items',index=False)
if len(sides)!=0:
    sides.to_excel(writer, sheet_name = 'sides',index=False)
if len(desserts)!=0:
    desserts.to_excel(writer, sheet_name = 'drinks-desserts',index=False)
writer.save()